In [2]:
# DEEP BOLTZMAN MACHINE (DBM)
import torch
import torch.nn as nn
class DBM(nn.Module):
    def __init__(self, vis_dim=784, hid_dims=[512, 256]):
        super().__init__()
        self.W1 = nn.Parameter(torch.randn(vis_dim, hid_dims[0]) * 0.1)
        self.W2 = nn.Parameter(torch.randn(hid_dims[0], hid_dims[1]) * 0.1)
    def sample(self, prob):
        return torch.bernoulli(prob)
    def forward(self, v):
        h1_p = torch.sigmoid(v @ self.W1)
        return h1_p
if __name__ == "__main__":
    vis_dim, lr, epochs = 784, 0.01, 5
    dbm = DBM(vis_dim)
    opt = torch.optim.Adam(dbm.parameters(), lr=lr)
    for epoch in range(epochs):
        data = torch.randint(0, 2, (32, vis_dim)).float()
        h1_p = dbm(data)
        if epoch == epochs - 1:
            print(f"Hidden Probabilities (h1_p) after training:\n", h1_p)


Hidden Probabilities (h1_p) after training:
 tensor([[0.0793, 0.2063, 0.0666,  ..., 0.8824, 0.7743, 0.0461],
        [0.0723, 0.4369, 0.0457,  ..., 0.9365, 0.8294, 0.2102],
        [0.3414, 0.2759, 0.0851,  ..., 0.9474, 0.8797, 0.2034],
        ...,
        [0.4538, 0.0902, 0.1899,  ..., 0.7619, 0.9514, 0.0483],
        [0.1530, 0.7704, 0.1735,  ..., 0.9808, 0.5619, 0.1016],
        [0.1556, 0.0370, 0.3908,  ..., 0.9281, 0.8220, 0.7774]],
       grad_fn=<SigmoidBackward0>)


In [3]:
# DEEP BELEIF NETWORK (DBN)
import torch
import torch.nn as nn
import torch.optim as optim
class DBN(nn.Module):
    def __init__(self, layer_sizes):
        super().__init__()
        self.layers = nn.ModuleList([nn.Linear(layer_sizes[i], layer_sizes[i+1]) for i in range(len(layer_sizes) - 1)])
    def forward(self, x):
        for layer in self.layers:
            x = torch.sigmoid(layer(x))
        return x
    def fine_tune(self, X, y, epochs=5, lr=0.01):
        self.fc = nn.Linear(self.layers[-1].out_features, len(torch.unique(torch.Tensor(y))))
        opt, loss_fn = optim.Adam(self.parameters(), lr=lr), nn.CrossEntropyLoss()
        data, targets = torch.Tensor(X), torch.LongTensor(y)
        for epoch in range(epochs):
            opt.zero_grad()
            loss = loss_fn(self.fc(self(data)), targets)
            loss.backward()
            opt.step()
        print("Weights after training (final layer):")
        print(self.fc.weight)
layer_sizes, X_train, y_train = [784, 512, 256, 128], torch.randn(100, 784), torch.randint(0, 10, (100,))
dbn = DBN(layer_sizes)
dbn.fine_tune(X_train, y_train, epochs=5, lr=0.01)


Weights after training (final layer):
Parameter containing:
tensor([[ 0.0565,  0.0402,  0.0540,  ...,  0.0549,  0.0512, -0.0398],
        [ 0.0616, -0.0143,  0.0519,  ...,  0.0077,  0.0090, -0.0641],
        [-0.0467, -0.0673,  0.0086,  ...,  0.0744,  0.0068,  0.0255],
        ...,
        [-0.0580, -0.0154, -0.0098,  ..., -0.0522,  0.0750,  0.0067],
        [-0.0738, -0.0395,  0.0099,  ..., -0.0206,  0.0788, -0.0191],
        [-0.0521, -0.0391, -0.0506,  ..., -0.0798, -0.0907, -0.0643]],
       requires_grad=True)
